In [0]:
!pip install rdflib
!pip install SPARQLWrapper
!pip install time

import rdflib as rb
from rdflib import Literal, BNode, URIRef, Graph, Namespace
from rdflib.namespace import RDF, OWL, RDFS, FOAF, XSD, DC
from SPARQLWrapper import SPARQLWrapper, JSON
import requests
from collections import OrderedDict
import pandas as pd
from time import sleep
import time


sch = rb.Namespace('http://schema.org/')
xsd = rb.Namespace('http://www.w3.org/2001/XMLSchema#')
rdfs = rb.Namespace('http://www.w3.org/2000/01/rdf-schema#')
owl = rb.Namespace('http://www.w3.org/2002/07/owl#')
ns1 = rb.Namespace('http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#')

In [0]:
g = Graph()
g.parse( "neighbourhood_crime_ontology.ttl", format = 'turtle')
g.bind('ns1', URIRef(ns1))
#sparql = SPARQLWrapper("http://localhost:3030/dataset.html?tab=upload&ds=/uoft-ds")

# sparql = SPARQLWrapper("http://localhost:3030/ds")

      #"http://localhost:3030/FinalResult/sparql"

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
######################################################################
#																	 #
# Define Functions with Sparql query to answer Competency Questions  #
#																	 #
######################################################################

url = "http://localhost:3030/FinalResult/sparql"

#CQ1: what is the ratio of occupations to crime?

def cq1(url):

  print("Please Input Community in the City of Toronto you Would Like to Explore")
  s1=input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

      SELECT ?comm (?sumppl/?crimecount AS ?occupationtocrimeratio)
      WHERE
      {
      {
      SELECT ?comm (SUM(?peoplecount) AS ?sumppl)
      WHERE 
      {
        ?communityID ns1:hasCategory ?labour;
                    ns1:community ?comm;
                    ns1:hasCharacteristic ?job;
                    ns1:hasValue ?people
       BIND(REPLACE(?people,",","") AS ?ppl)
       BIND(xsd:integer(?ppl) AS ?peoplecount)
       FILTER(REGEX(?comm,"%s"))
        FILTER(REGEX(?labour,"Labour"))
      }
      GROUP BY ?comm  
      }

      {
      SELECT (COUNT(?type) AS ?crimecount)
      WHERE 
      {
        ?case ns1:offenceOf ?type;
             ns1:committedInHood ?id.
       FILTER(REGEX(?id,"95"))
      }
      }
      }
      """ %s1)

  r = requests.get(url, params = {'format': 'json', 'query': query})
  data = r.json()
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')


In [0]:
#CQ2: What types of crimes, on average, are most committed over each month?

def cq2(url):

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

      SELECT DISTINCT ?month ?crimetype (COUNT(?crimetype) AS ?c)
      WHERE
      {
         ?case ns1:hasOccurMonth ?month;
             ns1:offenceOf ?crimetype.
      }
      GROUP BY ?month ?crimetype
      ORDER BY DESC(?c)
      """)
  
  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')

In [0]:
#CQ3: Which time interval has the highest occurrence of crime during a day?

def cq3(url):
  print("Please Input Which Day You Would Like to Know (choose from: Monday, Tuesday, Wednesday, Thursday, Firday, Saturaday or Sunday)")
  s1=input()
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s2=input()
  
  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

      SELECT ?hour (COUNT(?hour) AS ?crimecount)
      WHERE
      {
      ?case ns1:hasOccurDayOfWeek ?day;
      ns1:committedInNeighbourhood ?neighbourhood;
      ns1:hasOccurHour ?hour.
      FILTER(REGEX(?day, "%s"))
      FILTER(REGEX(?neighbourhood, "%s"))
      }
      GROUP BY ?hour
      ORDER BY ASC(?hour)
      LIMIT 10
      """%(s1,s2))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')



In [0]:
#CQ4: What types of crime occurred in a particular time interval?

def cq4(url):
  print("Please Input Which Day You Would Like to Know (choose from: Monday, Tuesday, Wednesday, Thursday, Firday, Saturaday or Sunday)")
  s1 = input()
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s2 = input()
  print("Pleas Indicate the Starting Hour of the Time Interval (input integer only)")
  s3 = input()
  print("Please Indicate the Ending Hour of the Time Interval (input integer only)")
  s4 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT DISTINCT ?hour ?type
      WHERE
      {
      	?case ns1:hasOccurDayOfWeek ?day;
           	  ns1:committedInNeighbourhood ?neighbourhood;
          	  ns1:hasOccurHour ?hour;
          	  ns1:offenceOf ?type.

        FILTER(REGEX(?day,"%s"))
        FILTER(REGEX(?neighbourhood,"%s"))
        FILTER(xsd:integer(?hour) >= %s)
        FILTER(xsd:integer(?hour) <= %s)
      }
      ORDER BY ASC(?hour)
      """%(s1,s2,s3,s4))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')

    

In [0]:
#CQ5: How many occurrences of a type of crime in a particular time interval?

def cq5(url):
  print("Please Input Which Day You Would Like to Know (choose from: Monday, Tuesday, Wednesday, Thursday, Firday, Saturaday or Sunday)")
  s1 = input()
  print("Please Input Which Month You Would Like to Know (choose from: January, February, March, April...and so on)")
  s2 = input()
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s3 = input()
  print("Pleas Indicate the Starting Hour of the Time Interval (input integer only)")
  s4 = input()
  print("Please Indicate the Ending Hour of the Time Interval (input integer only)")
  s5 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?hour ?type (COUNT(?type) AS ?ty)
      WHERE
      {
    	?case ns1:hasOccurDayOfWeek ?day;
            ns1:committedInNeighbourhood ?neighbourhood;
            ns1:hasOccurMonth ?month;
    	      ns1:hasOccurHour ?hour;
        	  ns1:offenceOf ?type.

      FILTER(REGEX(?day,"%s"))
      FILTER(REGEX(?month,"%s"))
      FILTER(REGEX(?neighbourhood,"%s"))
  
      FILTER(xsd:integer(?hour) >= %s)
      FILTER(xsd:integer(?hour) <= %s)
      }
      GROUP BY ?hour ?type
      ORDER BY ASC(?hour)

      """%(s1,s2,s3,s4,s5))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')


In [0]:
#CQ6: Has a type of crime increased or decreased between time interval 1 and time interval 2?

def cq6(url):
  print("Please Indicate the Time Interval 1 (choose from: January, February, March, April, ..., or December) ")
  s1 = input()
  print("Please Indicate the Time Interval 2 (choose from: January, February, March, April, ..., or December)")
  s2 = input()
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s3 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?month (COUNT(?type1) AS ?ty1)
      WHERE
      {
      	?case ns1:hasOccurMonth ?month;
              ns1:committedInNeighbourhood ?neighbourhood;
       	      ns1:offenceOf ?type1.

        FILTER(REGEX(?month,"%s")||REGEX(?month,"%s"))
        FILTER(REGEX(?neighbourhood,"%s"))
      }
      GROUP BY ?month

      """%(s1,s2,s3))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')


In [0]:
#CQ7: What neighborhood has the highest ratio of a type of crime to income?

def cq7(url):

  print("Pleas Indicate the Crime Type (choose from: [autoTheft], [breakAndEnter], [robbery], [assault], [theftOver])")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

      SELECT ?comm ?sal ?peoplecount (?crimecount/?peoplecount AS ?crimetypetoincomeratio)
      WHERE
      {{
      SELECT ?comm ?sal ?peoplecount
      WHERE 
      {
       ?communityID ns1:hasCategory ?income;
                    ns1:hasName ?comm;
                    ns1:hasCharacteristic ?sal;
                    ns1:hasValue ?people
        BIND(REPLACE(?people,",","") AS ?ppl)
        BIND(xsd:integer(?ppl) AS ?peoplecount)
        FILTER(REGEX(?comm,"Annex"))
        FILTER(REGEX(?income,"Income"))
       }
      GROUP BY ?comm ?sal ?peoplecount
      }
      {
      SELECT (COUNT(?type) AS ?crimecount)
      WHERE 
      {
       ?case ns1:offenceOf ?type;
             ns1:committedInNeighbourhood ?hood.
        FILTER(REGEX(?hood,"Annex"))
        FILTER(REGEX(?type,"%s"))
      }}}
      ORDER BY DESC(?crimetypetoincomeratio)
      """ % (s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result') 

In [0]:
#CQ8: What types of crime occurred most often in which area?

def cq8(url):
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?type1 ?lat ?long (COUNT(?type1) AS ?crimecount)
      WHERE
      {
    	?case ns1:committedInNeighbourhood ?neighbourhood;
       	    ns1:offenceOf ?type1;
  		      ns1:hasLatitude ?lat;
      	    ns1:hasLongtitude ?long
      FILTER(REGEX(?neighbourhood,"%s"))
      }
      GROUP BY ?type1 ?lat ?long
      ORDER BY DESC(?ty1)
      LIMIT 10

      """ %(s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')  

In [0]:
#CQ9: What type of housing has the most break-ins?

def cq9(url):
  print("Please Input Which Community in the City of Toronto you Would Like to Explore ")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT DISTINCT ?premisetype (COUNT(?type) AS ?ty)
      WHERE
      {
      	?case ns1:committedOnPremiseType ?premisetype;
              ns1:committedInNeighbourhood ?neighbourhood;
       	      ns1:offenceOf ?type.
      FILTER(REGEX(?type,"breakAndEnter"))
      FILTER(REGEX(?neighbourhood,"%s"))
      }
      GROUP BY ?premisetype

      """% (s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result') 


In [0]:
#CQ10: How many types of offence occurred in each neighborhood?

def cq10(url):
  print("Please Input Which Community in the City of Toronto you Would Like to Explore")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?neighbourhood ?type (COUNT(?type) AS ?ty)
      WHERE
      {
      	?case ns1:committedInHood ?neighbourhoodID;
              ns1:committedInNeighbourhood ?neighbourhood;
       	      ns1:offenceOf ?type.
  
        FILTER(REGEX(?neighbourhood,"%s"))
      }
      GROUP BY ?neighbourhood ?type
      ORDER BY DESC(?ty)

      """%(s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')

In [0]:
#CQ11: Which neighbourhood has the highest ratio of a type of crime to population age range?

def cq11(url):

  print("Pleas Indicate the Crime Type (choose from: [autoTheft], [breakAndEnter], [robbery], [assault], [theftOver])")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

      SELECT ?comm ?age ?peoplecount (?crimecount/?peoplecount AS ?crimetypetoincomeratio)
      WHERE
      {
      {
      SELECT ?comm ?age ?peoplecount
      WHERE 
      {
        ?communityID ns1:hasCategory ?pop;
                     ns1:community ?comm;
                     ns1:hasCharacteristic ?age;
                     ns1:hasValue ?people
        BIND(REPLACE(?people,",","") AS ?ppl)
        BIND(xsd:integer(?ppl) AS ?peoplecount)
        FILTER(REGEX(?comm,"Annex"))
        FILTER(REGEX(?pop,"Population"))
      }
      GROUP BY ?comm ?age ?peoplecount
      }
      {
      SELECT (COUNT(?type) AS ?crimecount)
      WHERE 
      {
       ?case ns1:offenceOf ?type;
             ns1:committedInNeighbourhood ?hood.
       FILTER(REGEX(?hood,"Annex"))
       FILTER(REGEX(?type,"%s"))
      }}}
      ORDER BY DESC(?crimetypetoincomeratio)
      """ % (s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result') 


In [0]:
#CQ12: What is the largest age group?

def cq12(url):

  print("Please Input Which Community in the City of Toronto you Would Like to Explore")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?age ?value
      WHERE {
      	?communityID ns1:hasName ?community;
                  	 ns1:hasCategory ?pop;
                		 ns1:hasCharacteristic ?age;
                		 ns1:hasValue ?value.
       FILTER(REGEX(?community,"%s"))
       FILTER(REGEX(?pop,"Population"))
       }
       ORDER BY DESC(xsd:integer(?value))

      """ %(s1))
 
  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result') 

In [0]:
#CQ13: What is the largest education level?

def cq13(url):
  print("Please Input Which Community in the City of Toronto you Would Like to Explore")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      Select ?edu ?value
      Where {
      	?communityID ns1:hasName ?community;
                  	 ns1:hasCategory ?ed;
		                 ns1:hasCharacteristic ?edu;
		                 ns1:hasValue ?value.
      FILTER(REGEX(?community,"%s"))
      FILTER(REGEX(?ed,"Education"))
      }
      ORDER BY DESC(xsd:integer(?value))
      """ %(s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')  


In [0]:
#CQ14: What is the distribution of educated people around each neighbourhood?

def cq14(url):

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      Select ?community ?edu ?value 
      Where {
      	?communityID ns1:hasName ?community;
      		           ns1:hasCategory ?ed;
		                 ns1:hasCharacteristic ?edu;
		                 ns1:hasValue ?value.
        FILTER(REGEX(?ed,"Education"))
      }
      ORDER BY ASC(?community)
      LIMIT 50
      """)

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')  



In [0]:
#CQ15: What is the mean salary range of each neighbourhood?

def cq15(url):
  print("Please Input Which Community in the City of Toronto you Would Like to Explore")
  s1 = input()

  query = ("""
      PREFIX ns1: <http://www.semanticweb.org/yiningzhou/ontologies/2019/11/NCA#>
      PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
      SELECT ?community ?salary ?val ((?v/?cnt1)*100 AS ?distribution)
      WHERE {
      	?communityID ns1:hasName ?community;
		                 ns1:hasCategory ?sal;
		                 ns1:hasCharacteristic ?salary;
  		               ns1:hasValue ?value.
  
        BIND(REPLACE(?value,",","") AS ?val)
        BIND(xsd:integer(?val) AS ?v)
        FILTER(REGEX(?community,"%s"))
        FILTER(?value !="nan")
        FILTER(REGEX(?sal,"Income"))
      {
      SELECT (SUM(?v) AS ?cnt1) 
      WHERE {
        	?communityID ns1:hasName ?community;
	                     ns1:hasCategory ?sal;
		                   ns1:hasCharacteristic ?salary;
  		                 ns1:hasValue ?value.      
      	  BIND(REPLACE(?value,",","") AS ?val)
          BIND(xsd:integer(?val) AS ?v)
          FILTER(?value !="nan")
          FILTER(REGEX(?community,"%s"))
          FILTER(REGEX(?sal,"Income"))
  		}
   }
}
ORDER BY ASC(?community)
LIMIT 50

      """ %(s1))

  try:
    r = requests.get(url, params = {'format': 'json', 'query': query})
    data = r.json()
  except requests.exceptions.ConnectionError:
    r.status_code = "Connection refused"
    
  result_dict= {}

  result_array=[]

  for item in data['results']['bindings']:
    
    for i in range(len(data['head']['vars'])):
        head = data['head']['vars'][i]
        result_dict.update(
            {head:item[head]['value']}
        )
        
    result_array.append(OrderedDict(result_dict))

  result = pd.DataFrame(linkedList)
  print(result)
  print('End of Result')   
  

In [0]:
######################################################################
#																	 #
# 						Main Program Start             	  		     #
#																	 #
######################################################################

while True:
	print("""Welcome to Neighbourhood Crime Analysis System!
	Please choose from one of the following options or "0" to exist:
  1. What is the ratio of occupations to crime?
  2. What types of crimes, on average, are most committed over each month?
  3. Which time interval has the highest occurrence of crime during a day? 
  4. What types of crime occurred in a particular time interval?
  5. How many occurrences of a type of crime in a particular time interval?
  6. Has a type of crime increased or decreased between time interval 1 and time interval 2?
  7. What neighborhood has the highest ratio of a type of crime to income? 
  8. What types of crime occurred most often in which area? 
  9. What type of housing has the most break-ins? 
  10. How many types of offence occurred in each neighborhood? 
  11. Which neighbourhood has the highest ratio of a type of crime to population age range?
  12. What is the largest age group?
  13. What is the largest education level?
  14. What is the distribution of educated people around each neighbourhood? 
  15. What is the mean salary range of each neighbourhood?
		""")
	url = "http://localhost:3030/FinalResult/sparql" 
	#take Keyboard input for selection
	try:
		choice = int(input())
	except ValueError:
		print("Please input a number from the available options")
		choice = int(input())

	#Execute query accordingly
	if choice == 0:
		print("Thanks for using Neighbourhood Crime Analysis System")
		break
	elif choice == 1:
		cq1(url)
	elif choice == 2:
		cq2(url)
	elif choice == 3:
		cq3(url)
	elif choice == 4:
		cq4(url)
	elif choice == 5:
		cq5(url)
	elif choice == 6:
		cq6(url)
	elif choice == 7:
		cq7(url)
	elif choice == 8:
		cq8(url)
	elif choice == 9:
		cq9(url)
	elif choice == 10:
		cq10(url)
	elif choice == 11:
		cq11(url)
	elif choice == 12:
		cq12(url)
	elif choice == 13:
		cq13(url)
	elif choice == 14:
		cq14(url)
	elif choice == 15:
		cq15(url)
